In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
USERNAME = "USERNAME"
PASSWORD = "PASSWORD"
PAGE = 1
# Initialize Chrome WebDriver
driver = webdriver.Chrome()
 
# Log in to Glassdoor
def login_to_glassdoor():
    driver.get("https://www.glassdoor.com/profile/login_input.htm")
    # Wait for page to loa
    time.sleep(2)
    # Find the username and password fields, and input your credentials

    username_field = driver.find_element(By.ID,"inlineUserEmail")
    username_field.send_keys(USERNAME)
    time.sleep(1)
    username_field.submit()
    print("1")
    time.sleep(1)
    # Click the login button
    # login_button = driver.find_element(By.CSS_SELECTOR, "button.email-button")
    # login_button.click()
    print("2")
    time.sleep(1)
    password_field = driver.find_element(By.ID,"inlineUserPassword")
    password_field.send_keys(PASSWORD)
    print("3")
    time.sleep(1)
    login_button = driver.find_element(By.CSS_SELECTOR, "button[data-size-variant='default']")
    login_button.click()
    print("4")
    # Wait for login to complete
    time.sleep(2)

 

# Navigate to the reviews page
def navigate_to_reviews():
    driver.get("https://www.glassdoor.com/Reviews/Honeywell-Reviews-E28_P" + str(PAGE) + ".htm")
    # driver.get("https://www.glassdoor.com/Reviews/Honeywell-Reviews-E28.htm")
    
    # Wait for page to load
    time.sleep(2)
 

# Extract reviews and save to a txt file
def save_reviews_to_txt(a):
    reviews = []
    for _ in range(10):
        try:
            # Find all review elements
            review_elements = driver.find_elements(By.CLASS_NAME,"noBorder.empReview.cf.pb-0.mb-0")
            # pros = driver.find_elements(By.XPATH,'//span[@data-test="pros"]')
            # review_elements = pros
            print("TRY:",a)
            # print(pros)
            for review_element in review_elements:
                try:
                    div_element = driver.find_element(By.CSS_SELECTOR,"div.v2__EIReviewDetailsV2__continueReading.v2__EIReviewDetailsV2__clickable.v2__EIReviewDetailsV2__newUiCta.mb")
                    div_element.click()
                except:
                    pass
                time.sleep(0.1)
                # print("ADDING SHIT")
                # print(review_element)
                review_text = review_element.text
                reviews.append(review_text)
                time.sleep(0.05)
                # print(review_text)
                # break
            break
        except:
            continue

    # Save the reviews to a txt file

    with open(str(a) + "glassdoor_reviews.txt", "w", encoding="utf-8") as file:
        file.write("\n\n".join(reviews))
    print("Reviews saved to glassdoor_reviews.txt")

 

def next_page():
    max_retries = 20
    for a in range(max_retries):
        try:
            button = driver.find_element(By.CSS_SELECTOR,"button[aria-label='Next']")
            button.click()
            time.sleep(0.2)
            return
        except:
            try:
                button = driver.find_element(By.CLASS_NAME,"qual_x_close")
                button.click()
            except:
                pass
            continue
    raise ValueError('no next page code stupid')
# Execute the entire process
def main():
    login_to_glassdoor()
    navigate_to_reviews()
    for a in range(PAGE,1000000000):
        save_reviews_to_txt(a)
        next_page()
        time.sleep(1)
    driver.quit()
 
if __name__ == "__main__":
    main()

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
# Initialize the WebDriver
driver = webdriver.Chrome()  # or webdriver.Firefox(), or any other WebDriver you're using

# Open the webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(2)
# Find the div element using XPath
parent_div = driver.find_element(By.CSS_SELECTOR, ".ag-center-cols-container")
# div_element = parent_div.find_elements(By.TAG_NAME, "div")
div_element = parent_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
# child_divs = viewport_div.find_elements_by_css_selector('.ag-center-cols-container > div')
# print(div_element.text)
# print(div_element.get_attribute('row-id'))

for a in div_element:
    print(a.text)
    temp = a.get_attribute('row-id')
    if temp is not None:
        print(a.get_attribute('row-id'))

# Do something with the div_element, for example, get its text


# Close the WebDriver
driver.quit()

              o5.5 -128
u5.5 +100
    o5.5 -130
u5.5 +100
o5.5 -130
u5.5 +100
        o5.5 -130
u5.5 +100
    
4360-58269
o19.5 -102
u19.5 -130
o19.5 -102
u19.5 -130
  o19.5 -105
u19.5 -125
o19.5 -112
u19.5 -112
o18.5 -122
u18.5 +100
o19.5 -105
u19.5 -125
o19.5 -120
u19.5 -110
o19.5 +110
u19.5 -152
o19.5 -110
u19.5 -120
o18.5 -123
u18.5 -105
o18.5 -123
u18.5 -105
o19.5 -108
u19.5 -124
o19.5 -103
u19.5 -130
o19.5 -133
u19.5 -118
o19.5 -103
u19.5 -130
o18.5 -123
u18.5 -105
o19.5 -108
u19.5 -124
o21.5 +155
u21.5 -235
5424-58269
      o7.5 -115
u7.5 -115
o7.5 -112
u7.5 -112
o7.5 -105
u7.5 -115
o7.5 -110
u7.5 -120
o7.5 -111
u7.5 -118
  o7.5 -115
u7.5 -115
o7.5 -108
u7.5 -120
o7.5 -108
u7.5 -120
        o7.5 -108
u7.5 -120
    
5603-58269
                                      
42526-58269
o26.5 -115
u26.5 -115
o26.5 -115
u26.5 -116
  o26.5 -115
u26.5 -115
o27.5 -112
u27.5 -112
o26.5 -122
u26.5 +100
o26.5 -115
u26.5 -115
o26.5 -110
u26.5 -120
o26.5 -114
u26.5 -120
o26.5 -115
u26.5 -115
o27.5 

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

def cleanpd(s):
    o = s.replace("  ", " ,").replace("\no", "\n,o").replace("\n ", "\n, ")
    return(o.split(","))

# Assuming you have set up your WebDriver, such as ChromeDriver
driver = webdriver.Chrome()

# Navigate to your webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(0.3)

# Find the first div with class .ag-center-cols-viewport
viewport_div = driver.find_element(By.CSS_SELECTOR,'.ag-center-cols-viewport')
name_div = driver.find_element(By.CSS_SELECTOR,'.ag-pinned-left-cols-container')

# Keep track of the number of child divs found
last_child_count = 0
player_data = {}
id2name = {}

# Loop to keep scrolling down until no more new child divs are loaded
while True:
    # Find all child divs under the viewport div with the specified CSS selector
    child_divs = viewport_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
    id_child = name_div.find_elements(By.CSS_SELECTOR, ".ag-pinned-left-cols-container > div")
    

    for index, div in enumerate(child_divs, start=1):
        # Get the text of the div
        try:
            temp = div.get_attribute('row-id')
            if temp is not None:
                # print(player_data)
                player_data[temp] = cleanpd(div.text)
        except:
            pass
    for index, div in enumerate(id_child, start=1):
        try:
            id2name[div.get_attribute('row-id')] = div.text
        except:
            pass
            
            
    if len(player_data.keys()) == last_child_count:
        break
    last_child_count = len(player_data.keys())
        # Print or process the text and attributes as needed


# Iterate through each child div to get its text and attributes


# Finally, remember to close the browser window when done
driver.quit()

IN DIV
2/13
7:30 PM
Luguentz Dort(G)
OKC
@
ORL
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Josh Giddey(G)
OKC
@
ORL
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Paolo Banchero(F)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Jonathan Isaac(F)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Markelle Fultz(G)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Kenrich Williams(G-F)
OKC
@
ORL
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Shai Gilgeous-Alexander
OKC
@
ORL
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Gary Harris(G)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Jalen Suggs(G)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Franz Wagner(F)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Aaron Wiggins(G)
OKC
@
ORL
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Moritz Wagner(F-C)
ORL
vs.
OKC
Edit
Projections
Start Trial

IN DIV
2/13
7:30 PM
Wendell Carter(C)
ORL
vs.
OKC
Ed

In [37]:
import numpy as np
arr = np.array([[1, 10, 9, 7], [3, 8, 2, 9], [7, 3, 4, 2], [5, 1, 2, 15]])


## 1. Sum all of the elements in the array "arr"
sum = np.sum(arr)

## 2. Find the mean of all elements in the array "arr"
mean = np.average(arr)

## 3. Reshape "arr" into a 2x8 matrix
reshaped = arr.reshape(2,8)

## 4. Create another array "arr2" of the same shape as "arr" with random numbers between 1 and 10
arr = np.array([[1, 10, 9, 7], [3, 8, 2, 9], [7, 3, 4, 2], [5, 1, 2, 15]])
arr2 = np.random.randint(0, 21, arr.shape)


## 5. Concatenate "arr" and "arr2" into a matrix with the shape 4x4x2
arr3 = np.vstack((arr, arr2))

## 6. Find the indicies of the top 2 values in each row in "arr"
top_indices = np.argsort(arr, axis=1)[:, -2:]


import pandas as pd

data = {'Name': ['Alice', 'Bob', 'Charlie', 'David'],
        'Age': [25, 30, 35, 40],
        'Salary': [50000, 60000, 70000, 80000]}
df = pd.DataFrame(data)

## Write a code snippet to filter the df DataFrame to only include rows where the Salary is greater than 60000 and display it.

display(df[df["Salary"] > 60000])


,Name,Age,Salary
2,Charlie,35,70000
3,David,40,80000


,Name,Age,Salary
2,Charlie,35,70000
3,David,40,80000


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

def cleanpd(s):
    o = s.replace("  ", " ,").replace("\no", "\n,o").replace("\n ", "\n, ")
    return(o.split(","))

# Assuming you have set up your WebDriver, such as ChromeDriver
driver = webdriver.Chrome()

# Navigate to your webpage
driver.get("https://unabated.com/nba/props")
driver.execute_script("document.body.style.zoom='30%'")
time.sleep(0.3)

# Find the first div with class .ag-center-cols-viewport
viewport_div = driver.find_element(By.CSS_SELECTOR,'.ag-center-cols-container')
name_div = driver.find_element(By.CSS_SELECTOR,'.ag-pinned-left-cols-container')

# Keep track of the number of child divs found
last_child_count = 0
player_data = {}
id2name = {}

# Loop to keep scrolling down until no more new child divs are loaded
while True:
    # Find all child divs under the viewport div with the specified CSS selector
    child_divs = viewport_div.find_elements(By.CSS_SELECTOR, '.ag-center-cols-container > div')
    id_child = name_div.find_elements(By.CSS_SELECTOR, ".ag-pinned-left-cols-container > div")
    

    
    # Scroll down the page to load more content
    time.sleep(0.3)
    # Create an ActionChains object
    actions = ActionChains(driver)

    # Scroll the element into view
    actions.move_to_element(child_divs[-1])

    # Perform a scroll action (e.g., scroll down)
    try:
        actions.click_and_hold().move_by_offset(0, 100).release().perform()
    except:
        pass

            
    if len(player_data.keys()) == last_child_count:
        break
    last_child_count = len(player_data.keys())
        # Print or process the text and attributes as needed


# Iterate through each child div to get its text and attributes


# Finally, remember to close the browser window when done
driver.quit()

In [5]:
# q = 'Bookmaker, Sharp Book, Sporttrade, bet365, pick6, Fanduel, DraftKings, BetMGM, Caesars, POINTSBET, BETONLINE, BOVADA, UNIBET, Sugar House, 888sport, BETRivers, Sports Betting, FourWinds, parx'.split(",")
# player_data["Source"] = q.split()
# for a in player_data.keys():
#     print(a, len(a))

In [11]:
import pandas as pd

df = pd.DataFrame(player_data)

display(df.T)

ValueError: All arrays must be of the same length

In [7]:
q = 'Bookmaker, Sharp Book, Sporttrade, bet365, pick6, Fanduel, DraftKings, BetMGM, Caesars, POINTSBET, BETONLINE, BOVADA, UNIBET, Sugar House, 888sport, BETRivers, Sports Betting, FourWinds, parx, '.split(",")

tempd = {}
for a in range(len(q)):
    tempd[a] = q[a]
tempd

{0: 'Bookmaker',
 1: ' Sharp Book',
 2: ' Sporttrade',
 3: ' bet365',
 4: ' pick6',
 5: ' Fanduel',
 6: ' DraftKings',
 7: ' BetMGM',
 8: ' Caesars',
 9: ' POINTSBET',
 10: ' BETONLINE',
 11: ' BOVADA',
 12: ' UNIBET',
 13: ' Sugar House',
 14: ' 888sport',
 15: ' BETRivers',
 16: ' Sports Betting',
 17: ' FourWinds',
 18: ' parx',
 19: ' '}

In [8]:
combinedpd = {}
for a in player_data.keys():
    combinedpd[id2name[a]]= player_data[a]

df = pd.DataFrame(combinedpd)
df = df.rename(index=tempd, errors="raise")
# Splitting the rows


display(df.T)

ValueError: All arrays must be of the same length

In [9]:
import re
def extract_text(s):
    # Define a regular expression pattern to match the desired components
    pattern = r'o(\d+\.\d+)\s*(-?\d+)\nu(\d+\.\d+)\s*([-+]\d+)'

    # Use re.findall to find all occurrences of the pattern in the string
    matches = re.findall(pattern, s)

    # Extract the matched groups from each occurrence
    # results = [(float(match[0]), int(match[1]), match[2], int(match[3])) for match in matches]
    if len(matches) == 0:
        return []
    # print(matches[0])
    return [float(matches[0][0]),int(matches[0][1]),float(matches[0][2]),int(matches[0][3])]

def oavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "o" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[1])
            else:
                o[results[0]] = [results[1]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2

def uavgc(row):
    o = {}
    o2 = {}
    for c in df.columns:
        if "u" in row[c]:
            results = extract_text(row[c])
            if len(results) != 4:
                continue
            if results[0] in o:
                o[results[0]].append(results[3])
            else:
                o[results[0]] = [results[3]]
    for p in o.keys():
        o2[p] = round(sum(o[p]) / len(o[p]),1)
    return o2


In [10]:
df['Over'] = df.apply(oavgc, axis=1)
df['Under'] = df.apply(uavgc, axis=1)
df[["Over", "Under"]].to_csv('out.csv')


NameError: name 'df' is not defined

In [110]:
list(df.iloc[1])

['o9.5 -126\nu9.5 -105\n',
 'o10.5 +100\nu10.5 -132\n',
 'o22.5 -112\nu22.5 -121\n',
 'o7.5 -146\nu7.5 +108\n',
 'o31.5 -110\nu31.5 -120\n',
 'o10.5 -127\nu10.5 -106\n',
 'o21.5 -127\nu21.5 -105\n',
 'o11.5 +100\nu11.5 -133\n',
 'o18.5 -124\nu18.5 -109\n',
 'o14.5 -164\nu14.5 +122\n',
 'o13.5 -103\nu13.5 -129\n',
 'o14.5 -129\nu14.5 -103\n',
 ' ',
 'o22.5 -117\nu22.5 -113\n',
 'o29.5 +105\nu29.5 -141\n',
 ' ',
 'o22.5 -125\nu22.5 -106\n',
 'o19.5 -129\nu19.5 -104\n',
 'o20.5 -140\nu20.5 +106\n',
 'o12.5 +103\nu12.5 -137\n',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'o12.5 -127\nu12.5 -106\n',
 'o23.5 -141\nu23.5 +107\n',
 'o26.5 -109\nu26.5 -122\n',
 'o13.5 -113\nu13.5 -119\n',
 'o11.5 -133\nu11.5 +100\n',
 'o19.5 -106\nu19.5 -125\n',
 ' ',
 'o29.5 +101\nu29.5 -137\n',
 'o10.5 -123\nu10.5 -108\n',
 'o10.5 -112\nu10.5 -121\n',
 'o19.5 -121\nu19.5 -111\n',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']